In [2]:
!pip install numpy==1.24.2
!pip install pandas==1.5.3
!pip install tabulate==0.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 10.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.8.10
    Uninstalling tabulate-0.8.10:
      Successfully uninstalled tabulate-0.8.10


In [7]:
import pandas as pd
df = pd.read_csv('lab1_ex01.csv', sep=',')

FileNotFoundError: [Errno 2] No such file or directory: 'lab1_ex01.csv'

In [1]:
import json

# Fileproj1_ex01.csvis a properly formed CSV file, with fields separated using commas (,) andwith column headers. Load it into a DataFrame.Create a file calledex01_fields.json, which contains informationallof the columns in the fileyou read. The file should contain an array of dictionaries with the following items:•column name (key:name),•percentage of missing values (key:missing, values in the range[0.0, 1.0]),•data type as a string with the following values (key:type:–intfor integer types,–floatfor floating-point types,–otherfor all other types.


def prepare_json_from_dataframe(df, filename):
    columns = []
    for col in df.columns:
        col_info = {}
        col_info["name"] = col
        col_info["missing"] = df[col].isnull().sum() / len(df)

        if df[col].dtype == "int":
            col_info["type"] = "int"
        elif df[col].dtype == "float":
            col_info["type"] = "float"
        else:
            col_info["type"] = "other"

        columns.append(col_info)

    with open(filename, 'w') as outfile:
        json.dump(columns, outfile)

prepare_json_from_dataframe(df, "ex01_fields.json")

NameError: name 'df' is not defined

In [57]:
# Compute statistics for all columns in your dataframe.For numeric columns include:•the count of non-empty values (count),2 •the average (mean),•the standard deviation (std),•the minimum (min) and maximum (max) values,•the the 25th, 50th, and 75th percentiles (attribute names:25%,50%and75%, respectively).For non-numeric columns include:•the count of non-empty values (count),•the number of unique values (unique),•the most common value (top) and its frequency (number of occurences;freq).Save the result to a JSON file calledex02_stats.jsonwhich contains a dictionary at the top level;the keys in the dictionary are column names, and the values are dictionaries with keys as describedabove

def prepare_detailed_json_from_dataframe(dt, filename):
    stats_df = df.describe(include="all") # Compute summary statistics for all columns
    stats_df = stats_df.dropna(axis=1, how="all") # Remove columns with all null values
    df_dict = stats_df.to_dict() # Convert to dictionary
    for column in df_dict:
        df_dict[column] = {key: value for key, value in df_dict[column].items() if not pd.isnull(value)} # Remove null values from dictionary

    with open(filename, 'w') as outfile:
        json.dump(df_dict, outfile)

prepare_detailed_json_from_dataframe(df, "ex02_stats.json")

In [13]:
import re

# Rename (“normalize”) the columns in the dataframe, so that they (sort of) follow thePEP 8guidelines forvariable names.Apply the following rules:•keep only characters which belong to the[A-Za-z0-9_ ]class (capital and small letters,digits, underscore and space),•convert all letters to lowercase,•replace all spaces with underscores (_).Make the changes in your DataFrame persistent.

for col in df.columns:
    new_col = re.sub(r"[^A-Za-z0-9_ ]", "", col)
    new_col = new_col.lower()
    new_col = new_col.replace(" ", "_")
    df.rename(columns={col: new_col}, inplace=True)

df.to_csv("ex03_columns.csv", index=False)


In [14]:
#Create a JSON file called ex04_json.json, which contains an array of rows stored as dictionaries,each with the DataFrame columns as keys (and values as values, obviously)
df.to_json("ex04_json.json", orient="records")

#Create an MS Excel file called ex04_excel.xlsx, which contains the column headers, but not the index values.
df.to_excel("ex04_excel.xlsx", index=False)

#Create a pickle file called ex04_pickle.pkl with the DataFrame.
df.to_pickle("ex04_pickle.pkl")

In [15]:
import tabulate
#Load the DataFrame pickled in file lab1_ex05.pkl.Select the following items from the DataFrame:•the 2nd and 3rd columns (regardless of their names),•rows whose index values begin with the letter v.

ex05_df = pd.read_pickle("lab1_ex05.pkl")

result = ex05_df.iloc[:,[1,2]]
starts_with_v = result.index.str.startswith("v")
result = result[starts_with_v]

# Save the result to a Markdown table stored in file ex05_table.md. Include the result, but don’t put anything in cells with missing values (i.e. prevent nan from being printed there).
with open("ex05_table.md", 'w') as outfile:
    outfile.write(tabulate.tabulate(result.fillna(""), headers='keys', tablefmt='pipe', showindex=True, missingval=""))

# fillna("") is used to replace NaN with empty string

In [16]:
# Pandas DataFrames are two-dimensional structures. However, data in JSON files often has ahierarchical structure, e.g. objects (dictionaries) are nested within objects.Filelab1_ex06.jsoncontains an array with such hierarchical objects (the structure of each arrayelement is the same).Using the data in the file, create a Pandas DataFrame, which contains a flattened version of thedata. For nested dictionaries, the column names should have the keys separated using dots (.).

ex06_df = pd.read_json("lab1_ex06.json")
ex06_df = pd.json_normalize(ex06_df.to_dict(orient="records"))

ex06_df.to_pickle("ex06_pickle.pkl")

